In [1]:
import numpy as np

In [2]:
# Parametri degli esperimenti effettuati
architectures = ("4_4", "10_3")
sampling_methods = ("r", "km")
imgs_per_class = (10, 20, 40, 80)

In [3]:
# Funzione per il calcolo del numero di pesi e link di una singola BNN
def ensemble_comp_stats(architecture, sampling_method, n_imgs):
    layers = architecture.split("_")
    total_ensemble = 0
    links_ensemble = 0
    
    for i in range(10):
        for j in range(i+1, 10):
            folder_out = "../results/ensemble/gd_t/%d_%d/%s/%d/"%(int(layers[0]), int(layers[1]), sampling_method, n_imgs)
            file_out = folder_out + "bnn%d%d_%d_%d_%d.npz"%(i, j, int(layers[0]), int(layers[1]), n_imgs)
            
            # Calcolo pesi e link della singola BNN dell'ensemble
            with open(file_out, "rb") as file:
                nn_data = np.load(file, allow_pickle=True)
                weights = [nn_data["w0"], nn_data["w1"], nn_data["w2"]]
                bias = [nn_data["b0"], nn_data["b1"], nn_data["b2"]]

                total_w = 0
                links = 0

                # Calcolo dei numero totale di pesi e dei collegamenti
                for wl in weights:
                    for wv in wl:
                        for w in wv:
                            links += np.abs(w)
                            total_w += 1

                            
                total_ensemble += total_w
                links_ensemble += links

    return total_ensemble, links_ensemble

In [4]:
stats = dict()

for arch in architectures:
    for method in sampling_methods:
        for imgs in imgs_per_class:
            key = arch + "_" + method + "_" + str(imgs)
            t, l = ensemble_comp_stats(arch, method, imgs)
            l_ratio = round(l/t*100, 2) 
            exp_stats = (t, l, l_ratio)
            stats[key] = exp_stats

In [5]:
print(stats)

{'4_4_r_10': (142020, 36484.0, 25.69), '4_4_r_20': (142020, 39837.0, 28.05), '4_4_r_40': (142020, 42167.0, 29.69), '4_4_r_80': (142020, 44221.0, 31.14), '4_4_km_10': (142020, 30675.0, 21.6), '4_4_km_20': (142020, 32959.0, 23.21), '4_4_km_40': (142020, 34101.0, 24.01), '4_4_km_80': (142020, 36222.0, 25.5), '10_3_r_10': (354285, 91204.0, 25.74), '10_3_r_20': (354285, 99908.0, 28.2), '10_3_r_40': (354285, 105655.0, 29.82), '10_3_r_80': (354285, 110708.0, 31.25), '10_3_km_10': (354285, 76612.0, 21.62), '10_3_km_20': (354285, 82185.0, 23.2), '10_3_km_40': (354285, 85094.0, 24.02), '10_3_km_80': (354285, 90604.0, 25.57)}
